In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from new_service import get_information_elespectador


In [21]:

def obtener_articulos(text, paginas=100):
    """
    Extrae los nombres de los artículos, URLs y fechas de publicación de las páginas especificadas
    de la búsqueda en El Tiempo sobre migrantes venezolanos en Colombia.

    Parameters:
    text(str): Texto de busqueda del articulo
    paginas(int): Número de páginas a procesar.
    :return: DataFrame con las columnas 'Título', 'Fecha de Publicación' y 'URL'.
    """
    # Lista para almacenar los datos
    datos = []
    try:
        for page in range(paginas):
            if (page): 
                page = str(page) + '0'
            
            url_api = f'https://www.elespectador.com/pf/api/v3/content/fetch/searcherTag?query=%7B%22author%22%3Anull%2C%22date%22%3Anull%2C%22from%22%3A{page}%2C%22keyword%22%3A%22{text}%22%2C%22section%22%3A%5B%22%2Fcolombia%22%5D%2C%22subtype%22%3A%5B%22Art%C3%ADculos%22%5D%7D&d=937&_website=el-espectador'
            response = requests.get(url_api,timeout=10)
            response.raise_for_status()

            soup = BeautifulSoup(response.content, 'html.parser')
            html_content = soup.string.strip()

            data = json.loads(html_content)

            for element in data.get('content_elements', []):
                title = element.get('headlines', {}).get('basic', {})
                date = element.get('display_date', 'No Date')
                url = element.get('canonical_url', 'not found')
                url = 'https://www.elespectador.com' + url
                
                datos.append({
                    'title': title,
                    'date': date,
                    'url': url
                })

    except requests.exceptions.RequestException as e:
        print(f'Error al realizar la solicitud: {e}')
    finally:
        df = pd.DataFrame(datos)
        return df


In [23]:
articles = obtener_articulos('migrante-venezolano', paginas=100)

In [24]:
articles['content'] = None
articles['#coments'] = None
articles['coments'] = None
articles['tags'] = None
for index, row in articles.iterrows():
    content, coments, tags = get_information_elespectador(row['url'])
    # Agregar el contenido del articulo
    articles.loc[index, 'content'] = content
    if coments is not None:
        articles.loc[index, '#coments'] = len(coments)
        articles.loc[index, 'coments'] = coments
    if tags is not None:
        articles.loc[index, 'tags'] = tags

Error en la solicitud: 404 Client Error: Not Found for url: https://www.elespectador.com/pf/api/v3/content/fetch/comments?query=%7B%22articleId%22%3A%22BG3QUNLOOZB4PBMYSEUNAKJA34%22%7D&d=937&_website=el-espectador
Error en la solicitud: 404 Client Error: Not Found for url: https://www.elespectador.com/pf/api/v3/content/fetch/comments?query=%7B%22articleId%22%3A%22ASIKTZGV6VGRVIEGBGHGFEXGLM%22%7D&d=937&_website=el-espectador
Error en la solicitud: 404 Client Error: Not Found for url: https://www.elespectador.com/pf/api/v3/content/fetch/comments?query=%7B%22articleId%22%3A%22TGSGRFOLJBDQFGSWJN3VOSWL7Q%22%7D&d=937&_website=el-espectador
Error en la solicitud: 404 Client Error: Not Found for url: https://www.elespectador.com/pf/api/v3/content/fetch/comments?query=%7B%22articleId%22%3A%22MAZBMDRZB5CJJAEWXKN6DHSANY%22%7D&d=937&_website=el-espectador
Error en la solicitud: 404 Client Error: Not Found for url: https://www.elespectador.com/pf/api/v3/content/fetch/comments?query=%7B%22articleId%

In [26]:
articles

                                                title  \
0   Aumentó el número de migrantes que cruzaron la...   
1   Migración Colombia entregará permisos temporal...   
2   Asesinaron a tres migrantes venezolanos en Cas...   
3   Asesinaron a un extranjero en un hotel del cen...   
4   Habrá feria de empleo para venezolanos en Mede...   
..                                                ...   
82  Canciller pide crear fondo multilateral para t...   
83         El drama de ser migrante venezolano y LGBT   
84  Arquidiócesis de Cali dará alojamiento a 200 v...   
85  Gobierno y ONU iniciaron registro de migrantes...   
86  Cúcuta abre primer Centro de Atención Transito...   

                        date  \
0   2024-04-01T02:19:33.129Z   
1   2023-12-15T22:39:47.098Z   
2   2023-11-21T18:50:23.188Z   
3   2023-10-18T02:28:17.866Z   
4   2023-10-02T19:09:34.429Z   
..                       ...   
82      2018-09-02T15:19:12Z   
83      2018-08-30T03:00:00Z   
84      2018-07-31T20:45:44